In [1]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(101)

problem uzayi icin gerekli olan degerler

In [2]:
farkli_renk_miktari = 10

populasyon_sayisi = 50
secilen_birey_sayisi = 20

renkler_arasi_zaman_maks = 30
renkler_arasi_zaman_min = 5

her_jenerasyonda_yeni_birey_sayisi = 50

uzayin olusturulmasi

In [3]:
renkler_arasi_zaman = np.zeros((farkli_renk_miktari, farkli_renk_miktari))

for i in range(farkli_renk_miktari):
    for j in range(farkli_renk_miktari):
        if i <= j:
            zaman = np.random.randint(renkler_arasi_zaman_min, renkler_arasi_zaman_maks, 1)
            renkler_arasi_zaman[i][j] = zaman
            renkler_arasi_zaman[j][i] = zaman
        if i == j:
            renkler_arasi_zaman[i][i] = 2*renkler_arasi_zaman_maks

Genetik algoritma icin degerlerin tanimlanmasi

In [4]:
max_iterasyon = 1000

max_crossover_olasiligi = 0.7
min_crossover_olasiligi = 0.5

max_mutasyon_olasiligi = 0.6
min_mutasyon_olasiligi = 0.4

iterasyon sayisi ilerledikce crossover olasiligi azalip, mutasyon olasiligi artacak

Bunu yapmamizin sebebi, ilk iterasyonlarda exploration sonrasinda, buldundugumuz yerleri exploit etme istegimizdir. Crossover yapmak mutasyona gore uzayda bulundugumuz konumu daha fazla degistirecektir

In [5]:
def crossover_olasiligi(iterasyon):
    return max_crossover_olasiligi - (max_crossover_olasiligi-min_crossover_olasiligi)*iterasyon/max_iterasyon

In [6]:
def mutasyon_olasiligi(iterasyon):
    return min_mutasyon_olasiligi + (max_mutasyon_olasiligi-min_mutasyon_olasiligi)*iterasyon/max_iterasyon

constructive heuristic modeli ilk populasyonu belirlemek icin kullaniyoruz, bize daha iyi bir initial point veriyor

In [7]:
def greedy_sira():
    sira = np.zeros(farkli_renk_miktari)
    ilk_renk = np.random.randint(farkli_renk_miktari)
    sira[0] = ilk_renk
    
    greedy_renkler_arasi_zaman = renkler_arasi_zaman
    
    for i in range(1, farkli_renk_miktari):
        sira[i] = np.argmin(renkler_arasi_zaman[int(sira[i-1])])
        greedy_renkler_arasi_zaman[int(sira[i-1]),:] = 2*renkler_arasi_zaman_maks
        greedy_renkler_arasi_zaman[:,int(sira[i-1])] = 2*renkler_arasi_zaman_maks
    return sira

Sirada gecirilen sureyi minimalize etmek istedigimiz icin deger fonksiyonu olarak 1/gecen sure kullandik

In [8]:
def siranin_degeri(sira):
    deger = 0
    for i in range(farkli_renk_miktari-1):
        deger = deger + renkler_arasi_zaman[int(sira[i])][int(sira[i+1])]
    
    return 1/deger

greedy olarak elde ettigimiz siradan yeni bireyler elde etmek icin 2 Opt algoritmasini kullandik

In [9]:
def twoOptSwap(sira, i, k):
    yeni_sira = sira
    yeni_sira[i:k+1] = np.flip(sira[i:k+1])
    return yeni_sira

In [10]:
def populasyonun_olusturulmasi(sira):
    populasyon = np.zeros((populasyon_sayisi, farkli_renk_miktari))
    populasyon[0,:] = sira
    for i in range(populasyon_sayisi-1):
        j = np.random.randint(farkli_renk_miktari)
        k = np.random.randint(farkli_renk_miktari)
        if j > k:
            yeni_sira = twoOptSwap(sira, k, j)
        else:
            yeni_sira = twoOptSwap(sira, j, k)
        populasyon[i+1] = yeni_sira
    return populasyon

olusturulan populasyondan en iyi bireylerin secilmesi

In [11]:
def en_iyi_bireyler(populasyon, en_iyi_birey_sayisi):
    bireylerin_degerleri = np.zeros(len(populasyon))
    en_iyi_birey_populasyonu = np.zeros((en_iyi_birey_sayisi, farkli_renk_miktari))
    
    for i in range(len(populasyon)):
        bireylerin_degerleri[i] = siranin_degeri(populasyon[i])
    
    for i in range(en_iyi_birey_sayisi):
        index = np.argmax(bireylerin_degerleri)
        en_iyi_birey_populasyonu[i] = populasyon[index]
        bireylerin_degerleri[i] = 0
    
    return en_iyi_birey_populasyonu

genetic algoritmayi beslemek icin, populasyonumuza random olarak olusturulmus bireyler ekliyoruz

In [12]:
def random_populasyon_olusturulmasi():
    populasyon = np.zeros((populasyon_sayisi, farkli_renk_miktari))
    base_birey = np.arange(farkli_renk_miktari)
    for i in range(populasyon_sayisi):
        np.random.shuffle(base_birey)
        populasyon[i] = base_birey
    
    en_iyi_random_populasyon = en_iyi_bireyler(populasyon, secilen_birey_sayisi)
    return en_iyi_random_populasyon
    

Parentlari rulet yontemi ile belirledik

In [13]:
def ebeveyn_secimi(ebeveynler):
    ebeveyn_skorlari = np.zeros(len(ebeveynler))
    ebeveyn_skorlari[0] = siranin_degeri(ebeveynler[0])
    
    for i in range(1, len(ebeveynler)):
        ebeveyn_skorlari[i] = ebeveyn_skorlari[i-1] + siranin_degeri(ebeveynler[i])
        
    toplam_skor = ebeveyn_skorlari[-1]
    
    secim = np.random.uniform(0,toplam_skor)
    
    for j in range(len(ebeveynler)):
        if ebeveyn_skorlari[j] > secim:
            return ebeveynler[j]

crossover fonksiyonumuz asagidaki gibidir

In [14]:
def crossover(ebeveyn1, ebeveyn2, iterasyon):
    yeni_bireyler = np.zeros((2, farkli_renk_miktari))
    yeni_bireyler[0] = ebeveyn1
    yeni_bireyler[1] = ebeveyn2
    
    if np.random.uniform(0,1) < crossover_olasiligi(iterasyon):
        crossover_noktasi = np.random.randint(farkli_renk_miktari)
        
        for i in range(crossover_noktasi):
            index1 = np.where(yeni_bireyler[0] == ebeveyn2[i])
            temp1 = yeni_bireyler[0][index1]
            yeni_bireyler[0][index1] = yeni_bireyler[0][i]
            yeni_bireyler[0][i] = temp1
            
            index2 = np.where(yeni_bireyler[1] == ebeveyn1[i])
            temp2 = yeni_bireyler[1][index2]
            yeni_bireyler[1][index2] = yeni_bireyler[1][i]
            yeni_bireyler[1][i] = temp2
            
    return yeni_bireyler
        
        

In [15]:
def mutasyon(birey, iterasyon):
    mutasyon_bireyi = np.zeros((1,farkli_renk_miktari))
    
    if np.random.uniform(0,1) < mutasyon_olasiligi(iterasyon):
        index1 = np.random.randint(farkli_renk_miktari)
        index2 = np.random.randint(farkli_renk_miktari)
        
        temp = birey[index1]
        birey[index1] = birey[index2]
        birey[index2] = temp
    
    
    for i in range(farkli_renk_miktari):
        mutasyon_bireyi[0][i] = birey[i]
        
    return mutasyon_bireyi

heuristic method ile baslangic populasyonunun olusturulmasi

In [16]:
iterasyon = 1

In [17]:
baslangic_sirasi = greedy_sira()
baslangic_populasyon = populasyonun_olusturulmasi(baslangic_sirasi)
baslangic_populasyon_en_iyiler = en_iyi_bireyler(baslangic_populasyon, secilen_birey_sayisi)

Her iterasyonda populasyonumuza random olarak ekleyecegimiz bireyleri seciyoruz

In [18]:
eklenecek_populasyon = random_populasyon_olusturulmasi()
crossover_oncesi_populasyon = np.concatenate((baslangic_populasyon_en_iyiler, eklenecek_populasyon))

rulet yontemi ile parentleri belirlenerek yeni bireyler uretilir ve populasyona eklenir

In [19]:
yeni_jenerasyon = crossover_oncesi_populasyon
for i in range(her_jenerasyonda_yeni_birey_sayisi):
    ebeveyn1 = ebeveyn_secimi(crossover_oncesi_populasyon)
    ebeveyn2 = ebeveyn_secimi(crossover_oncesi_populasyon)
    
    yeni_jenerasyon = np.concatenate((yeni_jenerasyon, crossover(ebeveyn1, ebeveyn2, iterasyon)))

yeni jenerasyon bireylerin sadece en iyileri sag kalacak sekilde kirpilir 

In [21]:
yeni_jenerasyon = en_iyi_bireyler(yeni_jenerasyon, secilen_birey_sayisi)

yeni_jenerasyondaki_bireyler_mutasyona_ugratilir

In [22]:
for i in range(len(yeni_jenerasyon)):
    yeni_jenerasyon = np.concatenate((yeni_jenerasyon,mutasyon(yeni_jenerasyon[i], iterasyon)))
 
yeni_jenerasyon = en_iyi_bireyler(yeni_jenerasyon, secilen_birey_sayisi)

Yeni iterasyona gecildiginde, populasyona yeni bireyler eklenerek, crossover ve mutasyon uygulanacak

yapilacaklar

3 - bitis sartinin belirlenmesi ve iterasyonlarin yapilmasi